This notebook will be used mainly for the Coursera Capstone project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Introduction

During the Covid-19 pandemic, the gym sector has suffered a big drop in the revenues due to the restrictions put in place by governments to limit infections. 
Even though many fitness enthusiasts created their own home gym to keep on training during the lockdown phase, the gym represents in the heart of many, not only a place where you can train with other people, but also an environment where you can socialise and create meaningful relationships. Those relationships that the pandemic has taught us to be very important. 

# Business Problem

Having said that the gym sector has been hardly beaten by social restrictions in the last months, it's important to make good strategic decisions not to loose even more money. 
The purchase/rent of the building where the gym will be located, is definitely the biggest and the most important a gym owner has to make. Therefore, it is fundamental to take in consideration as many factors has possibile in the choice of the location, as from it will depend a large part of the future of the business. 
PumpIt, is an international company that owns gyms in all the major European capitals; their goal for 2021 is target the US market starting from New York City and then expanding in the West Coast. More precisely, they have picked Brooklyn to be the starting point for their expansion project in the US. 
With a surface of 180 km^2, Brooklyn is the second largest borough in New York City after Queens; scanning such a big area to find the best location for a new gym, is not an easy task. This is why the company has hired as consultants, a team of data scientist to be supported during the decision making process with meaningful insights coming from data. 
What consultants have been asked to do, is not only to understand which is the best area in Brooklyn where to open a new gym, but also to check the gym rating nearby the chosen area to understand a little bit more about the competition.

# Data

The Data that will be used for this analysis are mainly of two types: 

1. Data about New York City neighbourhoods and borough with their longitude and latitude to be able to create maps

2. Venues Data retrieved using the Footsquare API to explore and analyse neighbourhoods in New York City, more specifically those located in the Brooklyn borough